In [ ]:
# import libraries
import numpy as np
import pandas as pd
import glob
import time
import shutil # for moving files from ondirectory to other
import ast
import io
import os

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
#Content from Paper.
#Since we are not provided with properties, authors used for trainig their model. So we are collecting by ourself
#and also use the ones they provided in different tables, grpahs and for explanation.
# Permissions
permissions = ['READ_PHONE_STATE', 'READ_SMS', 'SEND_SMS', 'WRITE_SMS', 'RECEIVE_SMS', 'GET_ACCOUNTS', 
                  'ACCESS_NETWORK_STATE', 'GET_TASKS', 'WAKE_LOCK', 'CAMERA', 'ACCESS_FINE_LOCATION', 
                   'RECORD_AUDIO', 'READ_CONTACTS',
                  ]

# API_method_calls
API_method_calls = ['Landroid/telephony/TelephonyManager;->getDeviceId', 
                       'Landroid/telephony/TelephonyManager;->getSimSerialNumber', 
                       'Landroid/telephony/SmsManager;->sendTextMessage', 
                       'Lorg/apache/http/impl/client;->DefaultHttpClient',
                       'Landroid/telephony/TelephonyManager;->getLine1Number', 
                       'Landroid/content/ContentResolver;->delete', 'Landroid/ContentResolver;->query', 
                       'Ljava/net/HttpURLConnection;->connect', 'Ljava/net/URLConnection;->connect', 
                       'Ljava/net/HttpURLConnection;->connect', 
                      ]

In [ ]:
#Collected permission + permissions from paper(described above)
permissions_120 = ['READ_PHONE_STATE', 'READ_SMS', 'SEND_SMS', 'WRITE_SMS', 'RECEIVE_SMS', 'CREATE_USERS',
                   'ACCESS_NETWORK_STATE', 'GET_TASKS', 'WAKE_LOCK', 'CAMERA', 'ACCESS_FINE_LOCATION', 
                   'RECORD_AUDIO', 'READ_CONTACTS', 'BIND_WALLPAPER', 'REQUEST_DELETE_PACKAGES', 'CONTROL_VPN',
                   'READ_FILE', 'MODIFY_AUDIO_SETTINGS', 'READ_MMS', 'DEVICE_ADMIN_ENABLED', 'CONTROL_KEYGUARD',
                   'MOUNT_UNMOUNT_FILESYSTEMS', 'LISTEN_CELL_LOCATION', 'EXPAND_STATUS_BAR',
                   'ACCESS_EXTERNAL_STORAGE', 'BROADCAST_STICKY', 'MANAGE_OWN_CALLS',
                   'ACCESS_LOCATION_EXTRA_COMMANDS', 'REQUEST_INSTALL_PACKAGES', 'SET_ALARM', 'SET_WALLPAPER',
                   'INSTALL_SHORTCUT',
                   'SET_WALLPAPER_HINTS', 'RECEIVE_BOOT_COMPLETED', 'ACCESS_COARSE_LOCATION', 
                   'KILL_BACKGROUND_PROCESSES', 'READ_SYNC_STATS', 'INSTALL_PACKAGES', 'MODIFY_PHONE_STATE',
                   'ANDROID_ID', 'SET_PROCESS_FOREGROUND', 'GET_CONTENT', 'WIFI_LOCK', 'SET_WALLPAPER_HINTS',
                   'DISABLE_KEYGUARD', 'READ_LOGAS', 'BLUETOOTH', 'ACCESS_DOWNLOAD_MANAGER',
                   'CALL_PHONE', 'ACCESS_BACKGROUND_SERVICE', 'ACCESS_ASSISTED_GPS', 'START_ANY_ACTIVITY',
                   'ACCESS_ALL_DOWNLOADS', 'READ_SYNC_SETTINGS', 'RECEIVE_BOOT_COMP', 'CHANGE_WIMAX_STATE', 
                   'INSTALL_LOCATION_PROVIDER', 'GET_PACKAGE_SIZE', 'BROADCAST_PACKAGE_REMOVED',
                   'BLUETOOTH_ADMIN', 'DIAGNOSTIC', 'CHANGE_NETWORK_MODE', 'WRITE_DATA', 
                   'REQUEST_COMPANION_RUN_IN_BACKGROUND',
                   'ACCESS_WIFI_STATE', 'RECEIVE_WAP_PUSH', 'NFC', 'READ_PERMISSION',
                   'VIBRATE', 'ACCESS_BSCI_DRM', 'STORAGE_INTERNAL',
                   'REGISTER_CONNECTION_MANAGER', 'TRANSMIT_IR','WRITE_SYNC_SETTINGS', 'READ_LOGS', 
                   'FREEZE_SCREEN', 'GET_PASSWORD',
                   'CHANGE_BACKGROUND_DATA_SETTING', 'USE_FINGERPRINT', 'BROADCAST_SMS', 'BACKUP', 'FORCE_BACK',
                   'CAPTURE_AUDIO_OUTPUT', 'ADD_VOICEMAIL', 'CLEAR_APP_CACHE', 'CLEAR_APP_USER_DATA',
                   'WRITE_SECURE_SETTINGS', 'REQUEST_IGNORE_BATTERY_OPTIMIZATIONS', 'INTERNET', 'READ_CALL_LOG', 
                   'WRITE_CONTACTS', 'CHANGE_NETWORK_STATE', 
                   'READ_EXTERNAL_STORAGE', 'WRITE_EXTERNAL_STORAGE', 'ACCOUNT_MANAGER', 'USER_ACTIVITY',
                   'START_BACKGROUND_SERVICE', 'ACCESS_NETWORK_CONDITIONS', 'ASEC_DESTROY','READ_BLOCKED_NUMBERS',
                   'ACCESS_NOTIFICATION_POLICY', 'GET_ACCOUNTS', 'CALL', 'ACCESS_NOTIFICATIONS',
                   'USE_CREDENTIALS', 'READ_HISTORY_BOOKMARKS', 'REQUEST_COMPANION_USE_DATA_IN_BACKGROUND',
                   'BATTERY_STATS','CHANGE_WIFI_MULTICAST_STATE', 'REMOTE_AUDIO_PLAYBACK', 'READ_EXTERNAL_SOTRAGE', 'READ_CALENDAR',
                   'CHANGE_WIFI_STATE', 'REORDER_TASKS',
                   'ACCESS_LOCATION_EXTRA_COMMANDS','ACCESS_COSRSE_LOCATION', 'SET_TIME_ZONE', 'REMOVE_TASKS',
                   'CALL_PRIVILEGED', 'SET_ACTIVITY_WATCHER',]


In [ ]:
#Collected Sensitive API method API calls + API method calls from paper (described above)
API_method_calls_59 = ['Ljavax/sql/ConnectionEvent;-><init>', 'Ljava/nio/channels/WritableByteChannel;->close',
                       'Landroid/service/carrier/CarrierService;->stopSelf', 'Landroid/opengl/Matrix;->getClass',
                       'Landroid/view/ViewStructure;->setCheckable', 
                       'Landroid/text/method/BaseKeyListener;->getInputType', 
                       'Landroid/provider/MediaStore$Images$Media;->wait', 'Ljava/lang/Runtime;->exec', 
                       'Ljava/lang/System;->loadLibrary', 'Landroid/widget/AdapterView;->refreshDrawableState',
                       'Landroid/widget/MultiAutoCompleteTextView;->saveHierarchyState', 
                       'Ljava/io/BufferedOutputStream;-><init>', 'Ljava/io/FileOutputStream;-><init>',
                       'Landroid/app/PendingIntent;->send', 'Landroid/app/Activity;.<init>',
                       'Landroid/app/NativeActivity;->getVolumeControlStream', 
                       'Landroid/app/AcitivityManager;->killBackgroudProcess', 
                       'Landroid/content/pm/PacakageManager;->removePackageFromPrefe', 
                       'Ljava/util/Iterator;.next', 
                       'Landroid/content/pm/PacakageManager;->getInstalledPackages',
                       'Landroid/content/pm/PacakageManager;->getInstalledApplications', 
                       'Landroid/content/Intent;->startActivity', 'Landroid/content/Intent;->getAction',
                       'Ljava/lang/StringBuilder;.append', 'Landroid/content/ContentResolver;->delete', 
                       'Landroid/content/ContentResolver;->update', 'Landroid/content/ContentResolver;->insert', 
                       'Landroid/content/Context;->getPackageManager',
                       'Landroid/content/BroadcastReceiver;->abortBroadcast', 
                       'Landroid/database/sqlote/SQLiteDatabase;->execSQL',
                       'Landroid/media/MediaRecorder;->MediaRecorder', 'Ljava/net/HttpURLConnection;->connect', 
                       'Ljava/net/URLConnection;->connect', 'Ljavax/crypto/Cipher;->getInstance',
                       'Ljavax/crypto/Cipher;->Init', 'Ljavax/crypto/Cipher;->doFinal',
                       'Landroid/telephony/TelephonyManager;->getDataActivity', 
                       'Ljava/util/ArrayList;.<init>', 
                       'Landroid/telephony/TelephonyManager;->getLine1Number', 
                       'Ljava/lang/StringBuilder;.<init>', 
                       'Landroid/telephony/TelephonyManager;->getSubscriberId',
                       'Landroid/telephony/TelephonyManager;->getDeviceId',
                       'Landroid/telephony/TelephonyManager;->getNetworkType',
                       'Landroid/telephony/TelephonyManager;->getSimSerialNumber', 
                       'Landroid/telephony/TelephonyManager;->getSimState', 
                       'Landroid/telephony/SmsManager;->sendMultipartTextMessage',
                       'Landroid/telephony/SmsManager;->sendDataMessage', 
                       'Landroid/telephony/SmsMessage;->getServiceCenterAddress', 
                       'Landroid/telephony/gsm/SmsManager;->sendMultipartTextMessage',
                       'Ljava/lang/StringBuilder;.toString',
                       'Landroid/telephony/gsm/SmsManager;->getDisplayOriginatingAddress', 
                       'Landroid/telephony/PhoneStateListener;->onCallStateChanged',
                       'Lorg/apache/http/impl/client;->DefaultHttpClient',
                       'Ldalvik/system/DexClassLoader;->loadClass', 'Ldalvik/system/PathClassLoader;->loadClass', 
                       'Ljava/lang/Object;.<init>', 
                       'Landroid/net/wifi/WifiManager;->getConnectionInfo', 'Landroid/ContentResolver;->query',
                       'Landroid/location/LocationManager;->getLastKnownLocation']

In [ ]:
features_names = permissions_120 + API_method_calls_59
print(f'There are {len(features_names)} features to represent an APK in binary vector.')

There are 179 features to represent an APK in binary vector.


In [ ]:
#making numpy matrix for malware dataset (extracted permisssion and API calls).

malware_matrix = []
permissions_path = '/home/mujeeb/Adverserial_ML/AndroZoo/Malware/AndroGuard+Predefined_Dictionary/APKs_Properties_Files/APKs_Permissions/'
API_Calls_path = '/home/mujeeb/Adverserial_ML/AndroZoo/Malware/AndroGuard+Predefined_Dictionary/APKs_Properties_Files/APKs_API_Calls/'

APKs_shas = os.listdir(permissions_path)

for apk in APKs_shas[:4000]:
    apk_vector = np.zeros(179, dtype=int)
    permission_file = open(os.path.join(permissions_path + apk), 'r')
    apk_permissions = permission_file.read().splitlines()
    for perm in apk_permissions:
        if(perm in features_names): 
            apk_vector[features_names.index(perm)] = 1
    
    API_calls_file = open(os.path.join(API_Calls_path + apk), 'r')
    APK_api_calls = API_calls_file.read().splitlines()
    for api_call in APK_api_calls:
        if(api_call in features_names):
            apk_vector[features_names.index(api_call)]=1
            
    malware_matrix.append(apk_vector)

malware_matrix = np.array(malware_matrix)
np.save('/home/mujeeb/Adverserial_ML/AndroZoo/Malware/AndroGuard+Predefined_Dictionary/APKs_Properties_Files/Malware_Binary_Matrix_updated', malware_matrix)
print(malware_matrix.shape)

FileNotFoundError: ignored

In [ ]:
#making numpy matrix for benign dataset (extracted permissions and API_calls)

benign_matrix = []
permissions_path = '/home/mujeeb/Adverserial_ML/AndroZoo/Benign/Androguard+Predefined_Dictionary/APKs_Properties_Files/APKs_Permissions/'
API_Calls_path = '/home/mujeeb/Adverserial_ML/AndroZoo/Benign/Androguard+Predefined_Dictionary/APKs_Properties_Files/APKs_API_Calls/'

APKs_shas = os.listdir(permissions_path)

for apk in APKs_shas[:4000]:
    apk_vector = np.zeros(179, dtype=int)
    permission_file = open(os.path.join(permissions_path + apk), 'r')
    apk_permissions = permission_file.read().splitlines()
    for perm in apk_permissions:
        if(perm in features_names):
            apk_vector[features_names.index(perm)] = 1
    
    API_calls_file = open(os.path.join(API_Calls_path + apk), 'r')
    APK_api_calls = API_calls_file.read().splitlines()
    for api_call in APK_api_calls:
        if(api_call in features_names):
            apk_vector[features_names.index(api_call)]=1
            
    benign_matrix.append(apk_vector)

benign_matrix = np.array(benign_matrix)
np.save('/home/mujeeb/Adverserial_ML/AndroZoo/Benign/Androguard+Predefined_Dictionary/APKs_Properties_Files/Benign_Binary_Matrix_updated', benign_matrix)
print(benign_matrix.shape)

FileNotFoundError: ignored

### To perform attack  on 150 malware examples.

In [ ]:
#Benign permissions (having protection level 'Normal') and API calls mostly called by benign APKs.

normal_perm = ['ACCESS_LOCATION_EXTRA_COMMANDS', 'ACCESS_NETWORK_STATE','ACCESS_NOTIFICATION_POLICY',
                'ACCESS_WIFI_STATE', 'BLUETOOTH' ,'BLUETOOTH_ADMIN', 'BROADCAST_STICKY' ,'CHANGE_NETWORK_STATE',
                'CHANGE_WIFI_MULTICAST_STATE', 'CHANGE_WIFI_STATE', 'DISABLE_KEYGUARD', 'EXPAND_STATUS_BAR',
                'GET_PACKAGE_SIZE', 'INSTALL_SHORTCUT', 'INTERNET', 'KILL_BACKGROUND_PROCESSES', 'MANAGE_OWN_CALLS', 
                'MODIFY_AUDIO_SETTINGS', 'NFC', 'READ_SYNC_SETTINGS', 'READ_SYNC_STATS', 'RECEIVE_BOOT_COMPLETED', 
                'REORDER_TASKS', 'REQUEST_COMPANION_RUN_IN_BACKGROUND','REQUEST_COMPANION_USE_DATA_IN_BACKGROUND',
                'REQUEST_DELETE_PACKAGES', 'REQUEST_IGNORE_BATTERY_OPTIMIZATIONS', 'REQUEST_INSTALL_PACKAGES',
                'SET_ALARM' ,'SET_WALLPAPER', 'SET_WALLPAPER_HINTS', 'TRANSMIT_IR', 'USE_FINGERPRINT', 'VIBRATE',
                'WAKE_LOCK' ,'WRITE_SYNC_SETTINGS']
normal_API_Calls = ['Ljava/lang/Object;.<init>', 'Ljava/lang/StringBuilder;.toString', 
                    'Ljava/lang/StringBuilder;.append', 'Landroid/app/Activity;.<init>', 
                    'Ljava/lang/StringBuilder;.<init>', 
                    'Ljava/util/ArrayList;.<init>', 'Ljava/util/Iterator;.next',]



In [ ]:
normal_indecis = [features_names.index(feature) for feature in normal_perm+normal_API_Calls]
print(f'There are {len(normal_indecis)} benign features.')
print(f'Follwing are the indices of benign features where we can add these being features :\n', normal_indecis)

There are 43 benign features.
Follwing are the indices of benign features where we can add these being features :
 [27, 6, 100, 64, 46, 59, 25, 91, 108, 112, 44, 23, 57, 31, 88, 35, 26, 17, 66, 53, 36, 33, 113, 63, 106, 14, 87, 28, 29, 30, 32, 72, 78, 68, 8, 73, 175, 169, 143, 134, 159, 157, 138]


In [ ]:
from numpy import load
from random import randrange
normal_150 = load('/home/mujeeb/Adverserial_ML/AndroZoo/Malware/AndroGuard+Predefined_Dictionary/APKs_Properties_Files/150_malware_normal.npy')
print(normal_150.shape)

FileNotFoundError: ignored

In [ ]:
malicious_vector = normal_100[0]
print('APK binary vector before appending bening properties:\n', malicious_vector)

APK binary vector before changes:
 [1 0 0 0 0 0 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 1 1 0 0
 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 1 0 1 0 0 0 0 0 0 0 1 0 0 0 1 0 1 1 0 0
 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 1 1 0 0 0 0 0 0 0 0 0 1 0 1 1 1 1
 0 0 0 1 0 1 0 1 0 0 0 0 1 1 1 1 1 0 0 0 0 0 0 0 0 1 0 0 1 0 1]


In [ ]:
#Adding bening properties by making benign properties 0 to 1 with random position
for app in range(normal_100.shape[0]):
    perturb_percent = 7
    perturb_count = 0
    for i in range(len(normal_indecis)):
        rndm = normal_indecis[randrange(0,len(normal_indecis))]
        
        if(normal_100[app][rndm] == 0 and perturb_count < round(perturb_percent*1.79)):
            normal_100[app][rndm] = 1
            perturb_count +=1
            
print('APK binary vector after changes:')
print(normal_100[0])

APK binary vector after changes:
[1 0 0 0 0 0 1 1 1 1 1 1 0 0 1 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 1 1 0 1 1 0 1
 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 1 0 0 0 1 0
 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 1 0 1 0 0 0 0 0 0 1 1 0 0 0 1 0 1 1 0 0
 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 1 1 0 1 0 0 0 1 0 0 0 1 0 1 1 1 1
 0 0 0 1 0 1 0 1 0 1 0 1 1 1 1 1 1 0 0 0 0 1 0 0 0 1 0 1 1 0 1]


In [ ]:
#Adding bening properties by making benign properties 0 to 1 with left to right

for app in range(normal_100.shape[0]):
    perturb_percent = 7
    perturb_count = 0
    for i in normal_indecis:
        if(normal_100[app][i] == 0 and perturb_count < round(perturb_percent*1.79)):
            normal_100[app][i] = 1
            perturb_count +=1
            
print('APK binary vector after changes:')
print(normal_100[0])

APK binary vector after changes:
[1 0 0 0 0 0 1 1 1 1 1 1 0 0 0 0 0 1 0 0 0 1 0 1 0 1 1 1 0 0 0 1 0 1 1 1 0
 1 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 1 0 0 0 1 0 1 0 0 0 0 1 0 1 0 1 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 1 0 1 0 0 0 0 0 0 1 1 0 0 0 1 0 1 1 0 0
 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 1 1 0 0 0 0 0 0 0 0 0 1 0 1 1 1 1
 0 0 0 1 0 1 0 1 0 0 0 0 1 1 1 1 1 0 0 0 0 0 0 0 0 1 0 0 1 0 1]


In [ ]:
np.save('/home/mujeeb/Adverserial_ML/AndroZoo/Malware/AndroGuard+Predefined_Dictionary/APKs_Properties_Files/150_malware_attacked_8PerCent', normal_100)

###**Perform attack with Attack2**

In [ ]:
AEs_imges_path = '/home/mujeeb/Adverserial_ML/AEs_Generations/AEs_with_GANs/Generated_DCGAN/AEs/5xn/Equally_Distributed_Own_NN_PickingBenignPortion_top/AEs_12Percent_top/'
malware_APKs_path = '/home/mujeeb/Adverserial_ML/AndroZoo/Malware/AndroGuard+Predefined_Dictionary/Inter_Linear/11k_100_AEs_Testing/11k_100_APKs_AEs_Testing/*'
generated_images_path = '/home/mujeeb/Adverserial_ML/AEs_Generations/AEs_with_GANs/Generated_DCGAN/DCGAN_plesstan0_1/'

list_generated_images = os.listdir(generated_images_path)

for apk in glob.glob(malware_APKs_path):
    print('Analysing :', apk[-68:])

    
    #for nxn
    benign_with_maxConfidense = ['11k1000__0.jpg', '11k390__5.jpg', '11k490__9.jpg', '11k540__4.jpg', '11k840__5.jpg']#, '11k890__5.jpg', '11k920__4.jpg', '11k980__5.jpg', '11k330__0.jpg', '11k470__8.jpg']
    random_num = random.randint(0,4)
    random_img = os.path.join(generated_images_path + benign_with_maxConfidense[random_num])
    generated_benign = cv2.imread(random_img)
    
    #for 1xn, with max confidense
    #generated_benign = cv2.imread(os.path.join(generated_images_path + '11k270_1.jpg'))
    
    img = apk2image_equal(a,d,dx, generated_benign)
    #print(img.shape)
    
    #Saving to local directory
    cv2.imwrite(os.path.join(AEs_imges_path + apk[-68:-4] + '.jpg'), img)
    #print(os.path.join(malware_imges_path + apk[-68:-4] + '.jpg'))
    #except:
    #    pass